__Imports__

In [5]:
from pandas import read_csv

In [8]:
df = read_csv('data.csv')
df

,timestamp,activeWord,lastChar,correctChar,source,layout,testType,testLength,testLanguage,wildcards
0,2022-12-04T09:44:12.191Z,get,g,g,monkeytype,semicolemak,words,10,english,NaN
1,2022-12-04T09:44:12.368Z,get,e,e,monkeytype,semicolemak,words,10,english,NaN
2,2022-12-04T09:44:12.986Z,down,t,t,monkeytype,semicolemak,words,10,english,NaN
3,2022-12-04T09:44:13.154Z,down,,,monkeytype,semicolemak,words,10,english,NaN
4,2022-12-04T09:44:14.400Z,down,d,d,monkeytype,semicolemak,words,10,english,NaN
...,...,...,...,...,...,...,...,...,...,...
3724,2022-12-04T22:32:25.455Z,il,i,i,monkeytype,semicolemak,words,10,french,NaN
3725,2022-12-04T22:32:25.663Z,il,l,l,monkeytype,semicolemak,words,10,french,NaN
3726,2022-12-04T22:32:25.879Z,bas,,,monkeytype,semicolemak,words,10,french,NaN
3727,2022-12-04T22:32:27.530Z,bas,b,b,monkeytype,semicolemak,words,10,french,NaN
